Clone the repository

In [ ]:
!git clone https://github.com/AravindS1506/crowddiff.git

In [ ]:
%cd crowddiff

For visualization of results run the following script.Output crowd map after kde is stored at fused_kde_realization and density map is stored in density_map_kde

In [ ]:
!python kde_sample.py

If needed run the below scripts to generate a crowd map from the diffusion model. The following code uses the entire pipeline described in the paper

Install the requirements, incompatibility issues might appear, but  the required libraries will be installed

In [ ]:
!pip install -r requirements.txt


Refreshing might default the path to content, so ensure that the base is in the crowddiff folder

In [ ]:
%cd crowddiff

Install mpi4y, this command works in google colab

In [ ]:
!apt-get update && apt-get install -y libopenmpi-dev openmpi-bin
!pip install mpi4py

Preprocess jhu crowd dataset

In [ ]:
!python cc_utils/preprocess_jhu.py --data_dir  dataset --output_dir out_data --dataset jhu_crowd --mode test --image_size 256 --ndevices 1 --sigma '0.5'  --kernel_size '3'

Download the model and store it in the model folder

In [ ]:
%mkdir model
%cd model
import gdown

# Replace 'YOUR_FILE_ID' with the actual ID from your Google Drive link
file_id = '1dLEjaZqw9bxQm2sUU4I6YXDnFfyEHl8p'
gdown.download(f'https://drive.google.com/uc?id={file_id}', 'demo.pt', quiet=False)


# Replace 'FILE_ID' with the actual ID from your Google Drive link

Run the diffusion model code

In [ ]:
%cd ../
%cp -r "cc_utils" "scripts"

Output figures are stored in fused_kde_realization_pipeline.png

In [ ]:
DATA_DIR="--data_dir out_data/jhu_crowd/part_1/test/"
LOG_DIR="--log_dir results --model_path model/demo.pt"
TRAIN_FLAGS="--normalizer 0.8 --pred_channels 1 --batch_size 1 --per_samples 1"
MODEL_FLAGS="--attention_resolutions 32,16,8 --class_cond False --diffusion_steps 1000 --large_size 256  --small_size 256 --learn_sigma True --noise_schedule linear --num_channels 192 --num_head_channels 64 --num_res_blocks 2 --resblock_updown True --use_fp16 True --use_checkpoint True"
!CUDA_VISIBLE_DEVICES=0 python scripts/super_res_kde.py $DATA_DIR $LOG_DIR $TRAIN_FLAGS $MODEL_FLAGS